In [1]:
import numpy as np
import csv
import os
import math
import random
import pandas as pd
import matplotlib.pyplot as plt
random.seed(42)
np.random.seed(42)

In [2]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from statistics import mean
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [3]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPClassifier

In [4]:
from sklearn import linear_model

In [12]:
file_path= os.path.expanduser('~/EE219/hw4/insurance_data.csv')
df=pd.read_csv(file_path, sep=',',header='infer')

In [13]:
df

,ft1,ft2,ft3,ft4,ft5,ft6,charges
0,19,27.900,0,female,yes,southwest,16884.92400
1,18,33.770,1,male,no,southeast,1725.55230
2,28,33.000,3,male,no,southeast,4449.46200
3,33,22.705,0,male,no,northwest,21984.47061
4,32,28.880,0,male,no,northwest,3866.85520
5,31,25.740,0,female,no,southeast,3756.62160
6,46,33.440,1,female,no,southeast,8240.58960
7,37,27.740,3,female,no,northwest,7281.50560
8,37,29.830,2,male,no,northeast,6406.41070
9,60,25.840,0,female,no,northwest,28923.13692


In [14]:
def gender_to_num(days):
    convert = {'female': 0, 'male': 1}    
    num = []
    for i in range(0, len(days)):
        num.append(convert[days[i]])
    return num
def ft5_to_num(days):
    convert = {'yes': 0, 'no': 1}    
    num = []
    for i in range(0, len(days)):
        num.append(convert[days[i]])
    return num
def ft6_to_num(days):
    convert = {'southwest': 0, 'southeast': 1, 'northwest': 2, 'northeast': 3}    
    num = []
    for i in range(0, len(days)):
        num.append(convert[days[i]])
    return num

In [15]:
df['ft4'] =gender_to_num(df['ft4'])
df['ft5']=ft5_to_num(df['ft5'])
df['ft6']=ft6_to_num(df['ft6'])

In [26]:
df_y=df['charges']
df=df.drop(['charges'], axis = 1)

In [81]:
df_enc=df.drop(['ft1','ft2','ft3'], axis = 1)
enc = OneHotEncoder()
enc.fit(df_enc)
df_enc=enc.transform(df_enc).toarray()
df_enc.shape

F:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(1338, 8)

In [82]:
df_num=df.drop(['ft4','ft5','ft6'], axis = 1)

In [83]:
df_enc=np.append(df_num,df_enc, axis=1)

In [84]:
print(df_enc.shape)

(1338, 11)


In [86]:
df_enc

array([[19.  , 27.9 ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [18.  , 33.77,  1.  , ...,  1.  ,  0.  ,  0.  ],
       [28.  , 33.  ,  3.  , ...,  1.  ,  0.  ,  0.  ],
       ...,
       [18.  , 36.85,  0.  , ...,  1.  ,  0.  ,  0.  ],
       [21.  , 25.8 ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [61.  , 29.07,  0.  , ...,  0.  ,  1.  ,  0.  ]])

In [85]:
kf = KFold(n_splits=10,random_state=42,shuffle=True)
RMSE_train=[]
RMSE_test=[]
for train_index, test_index in kf.split(df):
    X_train= df_enc[train_index]
    y_train= df_y.loc[train_index]
    X_test= df_enc[test_index]
    y_test= df_y.loc[test_index]
#     print(X_train)
#     print(y_train)
    reg = LinearRegression()
    reg.fit(X_train, y_train)
    pred_train = reg.predict(X_train)
    pred_test = reg.predict(X_test)
    RMSE_train.append(mean_squared_error(y_train, pred_train))
    RMSE_test.append(mean_squared_error(y_test, pred_test))
print('RMSE for train data=',mean(RMSE_train))
print('RMSE for test data=',mean(RMSE_test))

RMSE for train data= 36474866.324134715
RMSE for test data= 37020264.84006275
